In [ ]:
# Install required packages in Google Colab
# !pip install -qq weaviate-client python-dotenv weaviate-agents datasets

# Weaviate Query Agent

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/scottaskinosie/weaviate_query_agent_demo/blob/main/weaviate_query_agent.ipynb)

Here, we'll walk you through a comprehensive example showcasing the **Weaviate Query Agent** functionality.  
The Query Agent is an intelligent layer that sits on top of your Weaviate vector database, using generative AI to optimize natural language queries and automatically determine the best search strategy.

### How the Query Agent Works

The Query Agent intelligently handles complex queries by:
- **Query Optimization**: Uses generative AI to transform natural language into optimized vector database queries
- **Collection Selection**: Automatically determines which collection(s) to search based on your question
- **Smart Filtering**: Decides when filtering is needed to narrow down results
- **Aggregation Logic**: Determines if aggregation operations should be performed (counts, grouping, etc.)
- **Execution**: Runs the optimized queries against your Weaviate instance

### Dataset Overview

This notebook demonstrates the Query Agent using three diverse collections:
- **Books**: 10,000 books with titles, authors, descriptions, and genres
- **Brands**: 104 clothing brands with descriptions, ratings, and company information  
- **Ecommerce**: 448 fashion items with prices, categories, reviews, and brand associations

All datasets come pre-vectorized with **Snowflake Arctic Embed v2.0** embeddings via Weaviate's embedding service.

### Further Reading

- Check out the datasets on [HuggingFace](https://huggingface.co/datasets/weaviate/agents) 
- Learn more about Query Agents in the [official documentation](https://docs.weaviate.io/agents/query)
- Explore the technical implementation in our [Query Agent tutorial](https://docs.weaviate.io/agents/query/tutorial-ecommerce)
- Understand vector databases in the [Weaviate developer docs](https://weaviate.io/developers/weaviate)

If running locally, to ensure smooth execution and prevent potential conflicts with your global Python environment, we recommend following the instructions in the README running the code in a virtual environment.

## Libraries/packages used 

The following libraries are used in this notebook:

* [<code style="color:blue;">weaviate-client[agents]:</code>](https://weaviate.io/developers/weaviate/client-libraries/python) A powerful vector database with Query Agent functionality for intelligent search
* [<code style="color:blue;">datasets:</code>](https://huggingface.co/docs/datasets/) Hugging Face datasets library for loading pre-vectorized data
* [<code style="color:blue;">os:</code>](https://docs.python.org/3/library/os.html) Used for environment variable management
* [<code style="color:blue;">dotenv:</code>](https://pypi.org/project/python-dotenv/) Loads environment variables from .env files

The packages mentioned above are already installed via the requirements.txt in the README. If you are running this in Colab or would like to manually install these packages, uncomment and run the first cell in this notebook:

* `pip install -qq weaviate-client python-dotenv weaviate-agents datasets`

<a id='TOC'></a>  
## Table of contents  

1. <a href="#Dependencies">Dependencies</a><br>
2. <a href="#Connecting">Connecting to your Weaviate cluster</a><br>
3. <a href="#Collections">Setting up collections and loading data</a><br>
   3.1 <a href="#BrandsCollection">Create and populate Brands collection</a><br>
   3.2 <a href="#EcommerceCollection">Create and populate Ecommerce collection</a><br>
   3.3 <a href="#BooksCollection">Create and populate Books collection</a><br>
4. <a href="#QueryAgent">Creating the Query Agent</a><br>
5. <a href="#ExampleQueries">Example Query Agent interactions</a><br>
   5.1 <a href="#ClassicRAG">Classic RAG search</a><br>
   5.2 <a href="#DatabaseFiltering">Generated database filtering</a><br>
   5.3 <a href="#StatisticalQueries">Statistical and aggregation queries</a><br>
   5.4 <a href="#MultipleQueries">Querying multiple databases</a><br>
   5.5 <a href="#MissingInfo">Identifying missing information</a><br>

<a id='Dependencies'></a>  
## 1. Dependencies
[Back to table of contents](#TOC)

This section initializes the necessary dependencies and sets environment variables for connecting to your Weaviate Cloud instance.

**If using Colab, add your keys between the quotation marks in the cell below**

In [ ]:
# un this cell to load your keys in Colab
import os

# set environment variables  
os.environ['WEAVIATE_URL'] = ''  # weaviate instance url  
os.environ['WEAVIATE_API_KEY'] = ''  # admin api key  

**If running locally, make sure your keys are in your .env file and run this cell**

In [1]:
# Run this cell if you are working locally in VS code or Code Space
import dotenv

dotenv.load_dotenv(override=True)

True

<a id='Connecting'></a>  
## 2. Connecting to your Weaviate cluster  
[Back to table of contents](#TOC)  

To interact with our Weaviate cluster, we'll initialize a client object and verify the connection is successful. This connection will be used throughout the notebook for all data operations and Query Agent interactions.

In [18]:
import weaviate, os

# Connect to Weaviate Cloud
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=os.getenv("WEAVIATE_URL"),
    auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WEAVIATE_API_KEY"))
)

# check if the connection is successful  
client.is_ready()

True

<a id='Collections'></a>  
## 3. Setting up collections and loading data  
[Back to table of contents](#TOC)  

In this section, we'll create three collections (Brands, Ecommerce, and Books) and populate them with pre-vectorized data from the [Weaviate Agents dataset on Hugging Face](https://huggingface.co/datasets/weaviate/agents). Each collection uses the Snowflake Arctic Embed v2.0 model for consistent vector embeddings.

<a id='BrandsCollection'></a>  
### 3.1 Create and populate Brands collection  
[Back to table of contents](#TOC)  

We'll start by creating the Brands collection with properties for clothing brand information, then populate it with brand records from the Weaviate agents dataset.

In [19]:
# import required module for configuration
import weaviate.classes.config as wc

# delete collection if it exists  
if client.collections.exists("Brands"):  
    client.collections.delete("Brands")  

client.collections.create(
    name="Brands", #Collection name

    #Set vectorizer configuration for text2vec using Weaviates embedding model
    vector_config=wc.Configure.Vectors.text2vec_weaviate(
        model="Snowflake/snowflake-arctic-embed-l-v2.0",
        source_properties=["name", "description", "child_brands", "parent_brand"],
    ),

)

print("Successfully created collection: Brands.")

Successfully created collection: Brands.


In [5]:
# Upload brands data using the Weaviate-recommended streaming approach
from datasets import load_dataset

# Load fresh streaming dataset for upload
brands_dataset = load_dataset("weaviate/agents", "query-agent-brands", split="train", streaming=True)

brands_collection = client.collections.get("Brands")
with brands_collection.batch.fixed_size(batch_size=100) as batch:
    for item in brands_dataset:
        batch.add_object(
            properties=item["properties"],
            vector=item["vector"]
        )

print("Successfully uploaded brands data using Weaviate streaming method!")

/Users/scottaskinosie/Documents/weaviate/weaviate_query_agent_demo/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Successfully uploaded brands data using Weaviate streaming method!


<a id='EcommerceCollection'></a>  
### 3.2 Create and populate Ecommerce collection  
[Back to table of contents](#TOC)  

Next, we'll create the Ecommerce collection for fashion items with properties including prices, categories, reviews, and brand associations, then load product records.

In [22]:
# import required module for configuration
import weaviate.classes.config as wc

# delete collection if it exists  
if client.collections.exists("Ecommerce"):  
    client.collections.delete("Ecommerce")  

client.collections.create(
    name="Ecommerce", #Collection name

    #Set vectorizer configuration for text2vec using Weaviates embedding model
    vector_config=wc.Configure.Vectors.text2vec_weaviate(
        model="Snowflake/snowflake-arctic-embed-l-v2.0",
        source_properties=["name", "description", "collection", "category", "subcategory", "brand", "colors", "tags", "reviews"],
    ),
)

print("Successfully created collection: Ecommerce.")

Successfully created collection: Ecommerce.


In [ ]:
# Upload ecommerce data using the Weaviate-recommended streaming approach
ecommerce_dataset = load_dataset("weaviate/agents", "query-agent-ecommerce", split="train", streaming=True)

ecommerce_collection = client.collections.get("Ecommerce")
with ecommerce_collection.batch.fixed_size(batch_size=100) as batch:
    for item in ecommerce_dataset:
        batch.add_object(
            properties=item["properties"],
            vector=item["vector"]
        )

print("Successfully uploaded ecommerce data using Weaviate streaming method!")

Successfully uploaded ecommerce data using Weaviate streaming method!


<a id='BooksCollection'></a>  
### 3.3 Create and populate Books collection  
[Back to table of contents](#TOC)  

Finally, we'll create the Books collection with properties for titles, authors, descriptions, and genres, then populate it with book records from various genres including mystery, fiction, and non-fiction.

In [21]:
# import required module for configuration
import weaviate.classes.config as wc

# delete collection if it exists  
if client.collections.exists("Books"):  
    client.collections.delete("Books")  

client.collections.create(
    name="Books", #Collection name

    #Set vectorizer configuration for text2vec using Weaviates embedding model
    vector_config=wc.Configure.Vectors.text2vec_weaviate(
        model="Snowflake/snowflake-arctic-embed-l-v2.0",
        source_properties=["title", "author", "description", "genres"],
    ),
)

print("Successfully created collection: Books.")

Successfully created collection: Books.


In [ ]:
# Upload books data using the Weaviate-recommended streaming approach
books_dataset = load_dataset("weaviate/agents", "query-agent-books", split="train", streaming=True)

books_collection = client.collections.get("Books")
with books_collection.batch.fixed_size(batch_size=100) as batch:
    for item in books_dataset:
        batch.add_object(
            properties=item["properties"],
            vector=item["vector"]
        )

print("Successfully uploaded books data using Weaviate streaming method!")

Successfully uploaded books data using Weaviate streaming method!


<a id='QueryAgent'></a>  
## 4. Creating the Query Agent  
[Back to table of contents](#TOC)  

Now we'll create our Query Agent instance, which will intelligently route queries across our three collections (Brands, Ecommerce, Books) and automatically determine the optimal search strategy.

The output from the Query Agent will include: the original query, the generated answer to the query, the searches performed, the collections searched, filters applied, aggregates completed, source objects pulled from the database that comntributed to the generated answer and any mising information if the generated answer is incomplete.

In [23]:
from weaviate_agents.query import QueryAgent

# Instantiate a new agent object, and specify the collections to query
qa = QueryAgent(
    client=client, 
    collections=["Brands", "Ecommerce", "Books"],  
)


<a id='ExampleQueries'></a>  
## 5. Example Query Agent interactions  
[Back to table of contents](#TOC)  

Let's explore the Query Agent's capabilities through various types of queries. Notice how the agent automatically determines which collections to search, applies filters, and performs aggregations based on the natural language input.

<a id='BookQueries'></a>  
### 5.1 Classic RAG search  
[Back to table of contents](#TOC)  

This Query Agent is able to respond as a classic RAG system. The Query Agent will run an initial generative step that will optimize the original query for a vector database.



In [ ]:
# Perform a query
response = qa.run(
    "Are there any books about King Arthur or Knights that I should read?"
)

# Print the response 
# The output will include: the original query, the generated answer to the query, the seraches performed,
# the collections searched, filters applied, aggregates completed and source objects ulled from the database.
response.display()

╭─────────────────────────────────────────────── 🔍 Original Query ───────────────────────────────────────────────╮
│                                                                                                                 │
│ Are there any books about King Arthur or Knights that I should read?                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📝 Final Answer ────────────────────────────────────────────────╮
│                                                                                                                 │
│ There are several highly recommended books about King Arthur and knights that you might enjoy. For King Arthur, │
│ notable titles include:                                                                                         │
│ - "Excalibur (The Warlord Chronicles, #3)" by Bernard Cornwell, which completes his trilogy with a fresh and    │
│ riveting take on Arthur's final struggles, blending elements of love, war, loyalty, and betrayal.               │
│ - "The Road to Avalon (Dark Ages of Britain, #1)" by Mary Jo Putney, a historical novel portraying King         │
│ Arthur’s court with a focus on intrinsic goodness and a moving romance involving Arthur and Morgan of Avalon.   │
│ - "The Once and Future King" by T.H. White, a classic and masterful retelling of the Arthurian legend combining │
│ comedy and tragedy to bring the British epic to life.                                                           │
│                                                                                                                 │
│ Regarding knights, you might like:                                                                              │
│ - "A Knight of the Seven Kingdoms (The Tales of Dunk and Egg, #1-3)" by George R.R. Martin, which is a fantasy  │
│ collection of novellas about the adventures of a young hedge knight and his squire in the world of Westeros,    │
│ filled with chivalry, royal intrigue, and epic exploits.                                                        │
│                                                                                                                 │
│ These selections cover a mix of historical fiction, fantasy, romance, and epic storytelling, giving you a       │
│ well-rounded experience of the legends and adventures surrounding King Arthur and knights.                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 1/2 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['books about King Arthur'],                                                                        │
│     filters=[[]],                                                                                               │
│     filter_operators='AND',                                                                                     │
│     collection='Books'                                                                                          │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 2/2 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['books about knights'],                                                                            │
│     filters=[[]],                                                                                               │
│     filter_operators='AND',                                                                                     │
│     collection='Books'                                                                                          │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 📊 No Aggregations Run                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📚 Sources ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  - object_id='3097f4cc-98d4-4264-bdad-e47cc04f3342' collection='Books'                                          │
│  - object_id='523376df-42b8-4471-9983-411e07fb6390' collection='Books'                                          │
│  - object_id='6e2af026-6e13-4d9d-b3b7-67ea08b1a435' collection='Books'                                          │
│  - object_id='6637ef4d-b063-49bf-8102-3e521a0f13ef' collection='Books'                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

   📊 Usage Statistics   
┌────────────────┬──────┐
│ LLM Requests:  │ 4    │
│ Input Tokens:  │ 9037 │
│ Output Tokens: │ 404  │
│ Total Tokens:  │ 9441 │
└────────────────┴──────┘

Total Time Taken: 12.57s

<a id='BookQueries'></a>  
### 5.2 Generated database filtering   
[Back to table of contents](#TOC)  

This Query Agent is able determine appropriate filters and apply filtering for database objects returned by the vector database query. This allows for dynamic filtering in RAG systems, a step that is normally a manual process.


In [12]:
# Perform a query
response = qa.run(
    "I am in the mood for a good mystery book, what should I read?"
)

# Print the response
response.display()

╭─────────────────────────────────────────────── 🔍 Original Query ───────────────────────────────────────────────╮
│                                                                                                                 │
│ I am in the mood for a good mystery book, what should I read?                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📝 Final Answer ────────────────────────────────────────────────╮
│                                                                                                                 │
│ If you're in the mood for a good mystery book, here are several strong recommendations from various authors and │
│ styles:                                                                                                         │
│                                                                                                                 │
│ 1. "The Beautiful Mystery" by Louise Penny – This is a captivating mystery set in a secluded monastery in       │
│ Quebec. The story revolves around a murder in a community of cloistered monks, blending a serene setting with   │
│ deep underlying tensions. This book has won prestigious awards like the Agatha, Anthony, and Macavity Awards    │
│ for best novel, promising a compelling read.                                                                    │
│                                                                                                                 │
│ 2. "A Good Girl's Guide to Murder" by Holly Jackson – This book offers a thrilling young adult perspective      │
│ where the protagonist re-investigates a closed murder case in her hometown, uncovering secrets that put her own │
│ safety at risk. It’s perfect if you enjoy suspenseful, contemporary mystery novels with a youthful angle.       │
│                                                                                                                 │
│ 3. "The Keepsake" by Tess Gerritsen – This novel combines forensic investigation with ancient mystery,          │
│ featuring a modern-day murder linked to a mummy in a museum. It’s a gripping read with suspense, crime, and     │
│ thriller elements, ideal if you like stories with scientific and archaeological intrigue.                       │
│                                                                                                                 │
│ 4. "Different Seasons" by Stephen King – Though known mainly for horror, this collection includes mystery and   │
│ thriller stories as well. It's a versatile choice if you appreciate well-crafted tales that blend genres.       │
│                                                                                                                 │
│ All these titles come highly recommended within the mystery genre and cater to different tastes, whether you    │
│ prefer classic detective work, young adult suspense, forensic thrillers, or a mix of horror and mystery. Choose │
│ based on what style intrigues you most!                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 1/2 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['good mystery books'],                                                                             │
│     filters=[                                                                                                   │
│         [                                                                                                       │
│             TextArrayPropertyFilter(                                                                            │
│                 property_name='genres',                                                                         │
│                 operator=<ComparisonOperator.CONTAINS_ANY: 'contains_any'>,                                     │
│                 value=['Mystery']                                                                               │
│             )                                                                                                   │
│         ]                                                                                                       │
│     ],                                                                                                          │
│     filter_operators='AND',                                                                                     │
│     collection='Books'                                                                                          │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 2/2 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['mystery genre books good reviews ratings'],                                                       │
│     filters=[                                                                                                   │
│         [                                                                                                       │
│             TextArrayPropertyFilter(                                                                            │
│                 property_name='genres',                                                                         │
│                 operator=<ComparisonOperator.CONTAINS_ANY: 'contains_any'>,                                     │
│                 value=['Mystery']                                                                               │
│             )                                                                                                   │
│         ]                                                                                                       │
│     ],                                                                                                          │
│     filter_operators='AND',                                                                                     │
│     collection='Books'                                                                                          │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 📊 No Aggregations Run                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📚 Sources ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  - object_id='4d04a10c-d9aa-463e-935f-82f7b30b38ed' collection='Books'                                          │
│  - object_id='ad97cb95-0cc3-45ec-a1e0-75d0c46f2aac' collection='Books'                                          │
│  - object_id='f860b3e8-d391-469c-a0b0-64249ebb643d' collection='Books'                                          │
│  - object_id='71f27cbe-ebbd-4175-aabb-b937906b870b' collection='Books'                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

   📊 Usage Statistics   
┌────────────────┬──────┐
│ LLM Requests:  │ 4    │
│ Input Tokens:  │ 9239 │
│ Output Tokens: │ 495  │
│ Total Tokens:  │ 9734 │
└────────────────┴──────┘

Total Time Taken: 10.66s

<a id='StatisticalQueries'></a>  
### 5.3 Statistical and aggregation queries  
[Back to table of contents](#TOC)  

The Query Agent automatically recognizes when aggregation is needed and performs complex statistical operations like counting and grouping data.

In [13]:
# Perform a query
response = qa.run(
    "Which author has the most books in my collection?"
)

# Print the response
response.display()

╭─────────────────────────────────────────────── 🔍 Original Query ───────────────────────────────────────────────╮
│                                                                                                                 │
│ Which author has the most books in my collection?                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📝 Final Answer ────────────────────────────────────────────────╮
│                                                                                                                 │
│ The author with the most books in your collection is Stephen King, with a total of 57 books. The next most      │
│ represented authors are William Shakespeare with 39 books and Terry Pratchett with 35 books. This indicates     │
│ that Stephen King has the largest representation in your book collection by count.                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 🔭 No Searches Run                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 📊 Aggregations Run 1/1 ────────────────────────────────────────────╮
│                                                                                                                 │
│ AggregationResultWithCollection(                                                                                │
│     search_query=None,                                                                                          │
│     groupby_property='author',                                                                                  │
│     aggregations=[                                                                                              │
│         TextPropertyAggregation(                                                                                │
│             property_name='title',                                                                              │
│             metrics=<TextMetrics.COUNT: 'COUNT'>,                                                               │
│             top_occurrences_limit=None                                                                          │
│         )                                                                                                       │
│     ],                                                                                                          │
│     filters=[],                                                                                                 │
│     collection='Books'                                                                                          │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

   📊 Usage Statistics    
┌────────────────┬───────┐
│ LLM Requests:  │ 3     │
│ Input Tokens:  │ 33637 │
│ Output Tokens: │ 164   │
│ Total Tokens:  │ 33801 │
└────────────────┴───────┘

Total Time Taken: 5.93s

<a id='MultipleQueries'></a>  
### 5.4 Querying multiple databases  
[Back to table of contents](#TOC)  

The Query agent looks at collection descriptions and property data to determine which collection or collections to query for proper context for the query response generation.

In [14]:
# Perform a query
response = qa.run(
    """I am looking to buy a cool new jean jacket or light coat at or below $150. 
    I want a trendy but also timeless look but can also hold up and last me a long time. 
    If possible, I want to purchase something from a new company that might get me a 
    sponsorship for my social media account."""
)
# Print the response
response.display()

╭─────────────────────────────────────────────── 🔍 Original Query ───────────────────────────────────────────────╮
│                                                                                                                 │
│ I am looking to buy a cool new jean jacket or light coat at or below $150.                                      │
│     I want a trendy but also timeless look but can also hold up and last me a long time.                        │
│     If possible, I want to purchase something from a new company that might get me a                            │
│     sponsorship for my social media account.                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📝 Final Answer ────────────────────────────────────────────────╮
│                                                                                                                 │
│ For a jean jacket or light coat under $150 that combines trendiness with a timeless look and durability, here   │
│ are some great options along with new brands that might offer sponsorship opportunities:                        │
│                                                                                                                 │
│ **Jean Jackets:**                                                                                               │
│ - The "Twilight Shimmer Jacket" by Vivid Verse ($90) has a trendy early 2000s Y2K style with a timeless baby    │
│ blue and subtle sheen, featuring a cropped fit and sleek zipper details. It is well-reviewed for style and      │
│ quality.                                                                                                        │
│ - The "Rustic Wanderer Utility Jacket" by Canvas & Co. ($120) offers rugged durability with weathered fabric    │
│ and multiple pockets, fitting a durable and lasting wear criterion.                                             │
│ - Another option is the "Garden Dusk Linen Jacket" by Loom & Aura ($90), a soft linen jacket with a timeless    │
│ cottagecore charm, though durability specifics aren't detailed.                                                 │
│                                                                                                                 │
│ **Light Coats:**                                                                                                │
│ - The "Elysian Quill Coat" by Loom & Aura ($150) is elegant, tailored with a vintage intellectual vibe, praised │
│ for its refined silhouette and intricate buttons - perfect for a timeless look.                                 │
│ - The "Ivory Parchment Trench" by Canvas & Co. ($150) is an elegant, classical-inspired trench coat in ivory,   │
│ suitable for spring and light outdoor wear.                                                                     │
│                                                                                                                 │
│ **Brands and Sponsorship Potential:**                                                                           │
│ - Loom & Aura is a newer brand offering jackets and coats in your price range with elegant, timeless looks.     │
│ - Canvas & Co. also fits the profile of producing durable and stylish outerwear.                                │
│ - Loom Legacy, a brand founded in 2020 in France and celebrated for timeless elegance and quality               │
│ craftsmanship, could be explored for sponsorship opportunities, although specific product listings were not     │
│ present in the search results.                                                                                  │
│                                                                                                                 │
│ While none of the exact items explicitly mention sponsorship offers, connecting with newer brands like Loom     │
│ Legacy, Loom & Aura, or Canvas & Co. via social media might open opportunities, especially since these brands   │
│ emphasize elegance, quality, and have a relatively recent foundation, suiting your criteria.                    │
│                                                                                                                 │
│ No detailed product specifically guarantees sponsorship but targeting new brands with active social engagement  │
│ could increase chances.                                                                                         │
│                                                                                                                 │
│ If durability beyond style detail is a top priority, th

╭─────────────────────────────────────────── 🔭 Searches Executed 1/8 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['jean jacket'],                                                                                    │
│     filters=[                                                                                                   │
│         [                                                                                                       │
│             IntegerPropertyFilter(                                                                              │
│                 property_name='price',                                                                          │
│                 operator=<ComparisonOperator.LESS_EQUAL: '<='>,                                                 │
│                 value=150.0                                                                                     │
│             )                                                                                                   │
│         ]                                                                                                       │
│     ],                                                                                                          │
│     filter_operators='AND',                                                                                     │
│     collection='Ecommerce'                                                                                      │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 2/8 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['light coat'],                                                                                     │
│     filters=[                                                                                                   │
│         [                                                                                                       │
│             IntegerPropertyFilter(                                                                              │
│                 property_name='price',                                                                          │
│                 operator=<ComparisonOperator.LESS_EQUAL: '<='>,                                                 │
│                 value=150.0                                                                                     │
│             )                                                                                                   │
│         ]                                                                                                       │
│     ],                                                                                                          │
│     filter_operators='AND',                                                                                     │
│     collection='Ecommerce'                                                                                      │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 3/8 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['trendy jean jacket timeless look'],                                                               │
│     filters=[[]],                                                                                               │
│     filter_operators='AND',                                                                                     │
│     collection='Ecommerce'                                                                                      │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 📊 No Aggregations Run                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────── ⚠️ Answer is Partial - Missing Information: ───────────────────────────────────╮
│                                                                                                                 │
│ - Specific info on sponsorship programs from these new brands                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📚 Sources ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  - object_id='e275768c-6ee4-4317-8e2c-e92cf8832173' collection='Ecommerce'                                      │
│  - object_id='d9b2a14b-22f3-423d-bc89-6c9e7e3718bd' collection='Ecommerce'                                      │
│  - object_id='c79f3154-3631-4501-8cef-8c32d63c14dd' collection='Ecommerce'                                      │
│  - object_id='dd6a71e8-47e8-4f50-a529-46ff48479908' collection='Ecommerce'                                      │
│  - object_id='148b8d80-c057-4d49-a617-689f7a0d6256' collection='Ecommerce'                                      │
│  - object_id='0c396681-3e27-4792-aa67-f7dd69281420' collection='Ecommerce'                                      │
│  - object_id='7b069490-d1b5-4906-aeb4-9f36dc82b70e' collection='Ecommerce'                                      │
│  - object_id='11ad330c-5650-442f-8e85-a0d0c67a6b78' collection='Ecommerce'                                      │
│  - object_id='66d2edd5-9dff-4d96-8b40-02cbe26f7805' collection='Brands'                                         │
│  - object_id='167fc961-aead-437d-a60a-e6c556665444' collection='Brands'                                         │
│  - object_id='648768e8-3fdb-481e-81d2-ff938c2d5841' collection='Brands'                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

   📊 Usage Statistics    
┌────────────────┬───────┐
│ LLM Requests:  │ 10    │
│ Input Tokens:  │ 27915 │
│ Output Tokens: │ 1082  │
│ Total Tokens:  │ 28997 │
└────────────────┴───────┘

Total Time Taken: 13.27s

<a id='MissingInfo'></a>  
### 5.5 Identifying missing information  
[Back to table of contents](#TOC)  

If a partial response is generated for a query the Query Agent will report the missing information not present in the database.

In [15]:
# Perform a query
response = qa.run(
    """I want to buy a new leather jacket but I am vegan and only buy from socially responsible brands."""
)
# Print the response
response.display()

╭─────────────────────────────────────────────── 🔍 Original Query ───────────────────────────────────────────────╮
│                                                                                                                 │
│ I want to buy a new leather jacket but I am vegan and only buy from socially responsible brands.                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📝 Final Answer ────────────────────────────────────────────────╮
│                                                                                                                 │
│ If you are looking for a vegan leather jacket from socially responsible brands, consider options such as the    │
│ "Shroud Jacket" by Vivid Verse. This jacket features a sleek, futuristic design using ultra-resistant fabric    │
│ that mimics leather but is likely synthetic and vegan-friendly. It is part of a collection with a strong        │
│ statement on style and protection, priced at $250 in colors black and silver. Another option is the "Rustic     │
│ Wanderer Utility Jacket" from Canvas & Co., which is durable, designed for outdoor use, and made from fabric    │
│ that aligns with a more rugged utility style, priced at $120. However, the data does not explicitly confirm if  │
│ these brands are certified socially responsible, but their products appear to cater to ethical and alternative  │
│ material use.                                                                                                   │
│                                                                                                                 │
│ Additionally, Velvet Vesper, a brand known for quality craftsmanship and timeless elegance from Japan, is noted │
│ for its social responsibility reputation, though it is not explicitly detailed to offer vegan leather jackets.  │
│ This brand could be explored further for vegan and socially responsible options.                                │
│                                                                                                                 │
│ In summary, vegan leather jackets by brands like Vivid Verse and Canvas & Co. exist, and Velvet Vesper is a     │
│ socially responsible brand worth considering, but direct confirmation of vegan leather and strict social        │
│ responsibility standards from these search results is not fully clear. You may want to research these brands    │
│ further to ensure they meet your specific criteria for vegan and social responsibility.                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 1/2 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['vegan leather jacket socially responsible brands'],                                               │
│     filters=[[]],                                                                                               │
│     filter_operators='AND',                                                                                     │
│     collection='Ecommerce'                                                                                      │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 2/2 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['vegan leather jacket socially responsible'],                                                      │
│     filters=[[]],                                                                                               │
│     filter_operators='AND',                                                                                     │
│     collection='Brands'                                                                                         │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 📊 No Aggregations Run                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────── ⚠️ Answer is Partial - Missing Information: ───────────────────────────────────╮
│                                                                                                                 │
│ - Explicit confirmation that the jackets are made from vegan leather material                                   │
│ - Detailed certification or accreditation of the brands as socially responsible                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📚 Sources ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  - object_id='406d684c-bb89-4aac-bd15-2bde1ad4ac54' collection='Ecommerce'                                      │
│  - object_id='11ad330c-5650-442f-8e85-a0d0c67a6b78' collection='Ecommerce'                                      │
│  - object_id='4a320459-cad7-4263-9db3-56e08611413b' collection='Brands'                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

   📊 Usage Statistics   
┌────────────────┬──────┐
│ LLM Requests:  │ 4    │
│ Input Tokens:  │ 8724 │
│ Output Tokens: │ 442  │
│ Total Tokens:  │ 9166 │
└────────────────┴──────┘

Total Time Taken: 10.59s

In [24]:
# close the Weaviate client to free up resources
client.close()